In [1]:
from __future__ import print_function
import librosa
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [2]:
def loadbeatmap(beatmap, num_beats, num_chunks_per_beat=8):
    if beatmap[len(beatmap)-5:len(beatmap)] != ".json":
        print("Beatmap file " + audio + " is not of type .json")
        return -1
    
    with open(beatmap) as f:
        data = json.load(f)
  
    notes = "_notes"
    time = "_time"
    line_index = "_lineIndex" #column number
    line_layer = "_lineLayer" #row number
    note_color = "_type" #0 is one color and 1 is the other
    cut_direction = "_cutDirection"#9 cut directions

    dim_0 = num_beats * num_chunks_per_beat
    
    # number of rows and columns in the playfield
    # number of cells in the playfield (each cell can hold at most 1 note)
    playfield_rows = 3
    playfield_cols = 4
    playfield_cell_count = playfield_rows * playfield_cols
    
    # number of colors (2): red, blue (order unknown)
    # number of directions notes can face (9): 
    # up, down, left, right, up-left, up-right, down-left, down-right, dot (order unknown)
    note_color_count = 2
    note_direction_count = 9
    
    # dimensions for a 'one-hot' representation of a single time unit (chunk)
    dim_1 = playfield_rows
    dim_2 = playfield_cols
    dim_3 = (note_color_count + 1) + (note_direction_count + 1)
    
    # initialize matrix to zeros, then set the "no note" bit for each block at each timestep to 1
    outMatrix = np.zeros(shape=(dim_0, dim_1, dim_2, dim_3))
    outMatrix[:,:,:,0] = 1
    outMatrix[:,:,:,3] = 1
    

    # for every note in the beatmap, set the color and direction bits for the proper cell to 1
    for n in range(len(data[notes])):
        entry = int(np.round(data[notes][n][time]*num_chunks_per_beat)) #convert time to row index by rounding to nearest 1/8 beat
        if data[notes][n][note_color] < 2:
            outMatrix[entry] \
                     [data[notes][n][line_layer]] \
                     [data[notes][n][line_index]] \
                     [data[notes][n][note_color]+1] = 1
            outMatrix[entry] \
                     [data[notes][n][line_layer]] \
                     [data[notes][n][line_index]] \
                     [0] = 0
            outMatrix[entry] \
                     [data[notes][n][line_layer]] \
                     [data[notes][n][line_index]] \
                     [data[notes][n][cut_direction]+4] = 1
            outMatrix[entry] \
                     [data[notes][n][line_layer]] \
                     [data[notes][n][line_index]] \
                     [3] = 0

    return outMatrix

In [3]:
def mean_center(x): 
    return (x - np.apply_along_axis(np.mean, 0, x) )

In [4]:
def loadsong(audio, samples_per_chunk=300, num_chunks_per_slice=65, num_chunks_per_beat=8, verbose=0):
    if audio[len(audio)-4:len(audio)] != ".ogg":
        print("Audio file " + audio + " is not of type .ogg")
        return -1
    
    y, sr = librosa.load(audio)
    
    song_length = librosa.get_duration(y=y,sr=sr) / 60.0
    tempo = np.round(librosa.beat.tempo(y, sr=sr))
    new_sample_rate = (tempo/200)*8000
    
    y = librosa.resample(y, sr, new_sample_rate)
    
    number_of_beats = int(tempo * song_length)
    
    return y[0:(len(y)//(samples_per_chunk*num_chunks_per_beat)*(samples_per_chunk*num_chunks_per_beat))], new_sample_rate, number_of_beats, tempo

In [5]:
def prep_song(song, sample_per_chunk = 300):
    song_y = song.reshape(len(song)//300,300)
    song_fft = np.abs(np.apply_along_axis(np.fft.fft, 1, song_y))[:,0:(int)(samples_per_chunk/2)+1]
    song_fft_mc = np.apply_along_axis(mean_center, 0, song_fft)
    return song_fft_mc

In [6]:
def append_song(init_song, init_beatmap, song_filepath, beatmap_filepath, num_chunks_per_beat = 8, num_beats_per_sequence = 32):
    num_chunks_per_sequence = num_chunks_per_beat * num_beats_per_sequence
    
    loaded_song_y, loaded_song_sr, num_beats, tempo = loadsong(song_filepath)
    
    prepped_song = prep_song(loaded_song_y)
    
    loaded_beatmap = loadbeatmap(beatmap_filepath, num_beats)
    
    if init_song == None and init_beatmap == None:
        init_song = []
        init_beatmap = []
        
    for i in range(num_beats*num_chunks_per_beat-num_chunks_per_sequence):
        init_song.append(prepped_song[i:i+num_chunks_per_sequence]) 
    for i in range(num_beats*num_chunks_per_beat-num_chunks_per_sequence):
        init_beatmap.append(loaded_beatmap[i:i+num_chunks_per_sequence]) 
    
    init_song = np.array(init_song)
    init_beatmap = np.array(init_beatmap)
        
    return init_song, init_beatmap, tempo

In [7]:
def data_prep(song_list, beatmap_list):
    X, Y, tempo = append_song(None, None, song_list[0], beatmap_list[0])
    for x, y in zip(song_list[1:], beatmap_list[1:]):
        X, Y, tempo = append_song(X, Y, x, y)
    
    return X, Y, tempo

In [8]:
def softmax_to_max(note_cell):
    output = []
    output.append(np.argmax(note_cell[:3]))
    output.append(np.argmax(note_cell[3:]))
    
    return output

In [9]:
#Thanks to @shouldsee from https://github.com/mpld3/mpld3/issues/434#issuecomment-340255689

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [17]:
def convert_beatmap_array(nn_out, division = 4):
    original_size = nn_out.shape[0]
    sequence_size = nn_out.shape[1]
    target_size = nn_out.shape[1] + nn_out.shape[0]

    slice_start = (sequence_size * (division - 1) // (division * 2))
    slice_end = sequence_size * (division + 1) // (division * 2)
    slice_size = slice_end - slice_start
    
    beatmap = np.zeros(shape=(target_size,3,4,13), dtype = float)
    #first section of the beatmap array
    beatmap[0:slice_end] = nn_out[0, 0:slice_end,:,:,:]
    
    #middle section
    i = slice_size
    while i < original_size:
        beatmap[i+slice_start:i+slice_end] = nn_out[i, slice_start:slice_end,:,:,:]
        i+=slice_size
    
    #final section 
    final_start_index = -(original_size - (i - slice_size))
    print(final_start_index)
    
    beatmap[final_start_index:target_size] = nn_out[original_size-1, final_start_index:sequence_size,:,:,:]
    
    return beatmap

In [32]:
def convert_to_json(original_beatmap, bpm, chunks_per_beat = 8, offset = 0.0, beats_per_bar=16, 
                    note_jump_speed=10, shuffle=0, shuffle_period=0.5, version="1.5.0"):
    new_beatmap = {}
    new_beatmap['_version'] = version
    new_beatmap['_beatsPerMinute'] = bpm
    new_beatmap['_beatsPerBar'] = beats_per_bar
    new_beatmap['_noteJumpSpeed'] = note_jump_speed
    new_beatmap['_shuffle'] = shuffle
    new_beatmap['_shufflePeriod'] = shuffle_period
    new_beatmap['_events'] = []
    new_beatmap['_notes'] = []
    new_beatmap['_obstacles'] = []

    
    new_beatmap['_notes'] = [
        {
            "_time" : (i  / chunks_per_beat) + offset,
            "_lineIndex" : k,
            "_lineLayer" : j,
            "_type" : original_beatmap[i][j][k][0] - 1,
            "_cutDirection" : original_beatmap[i][j][k][1] - 1
        } for i in range(original_beatmap.shape[0])
          for j in range(original_beatmap.shape[1])
          for k in range(original_beatmap.shape[2]) if original_beatmap[i][j][k][0] != 0]
    
    return new_beatmap

In [30]:
def export_beatmap(nn_output, tempo, filename = 'beatmap.json', 
                   division = 4, chunks_per_beat = 8, offset = 0.0,
                   beats_per_bar=16, note_jump_speed=10,
                   shuffle=0, shuffle_period=0.5, version="1.5.0"):
    
    resized_map = convert_beatmap_array(nn_output, division)
    converted_map = np.apply_along_axis(softmax_to_max, 3, resized_array)
    json_beatmap = convert_to_json(converted_map, tempo, chunks_per_beat, 
                                   offset, beats_per_bar, note_jump_speed, 
                                   shuffle, shuffle_period, version)

    with open(filename, 'w') as outfile:
        outfile.write(json.dumps(json_beatmap, cls=NumpyEncoder))

In [13]:
samples_per_chunk = 300
num_chunks_per_slice = 65
num_chunks_per_beat = 8

In [14]:
X, Y, tempo  = data_prep(["song.ogg"], ["Expert.json"]) #"song.ogg"], ["Expert.json", "Expert.json"])

/nfshome/apps/python-3.6.7/lib/python3.6/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


In [15]:
# Zube magic happens here
# But, for now, we just use te original matrix

In [24]:
# If we have issues where the net is predicting 'no note' for everything,
# I think we should try adding a sensitivity parameter and call a
# function here that lowers the softmax  value for all 'no note' predictions
# by some amount, giving the others a higher chance of being the max value

In [33]:
export_beatmap(Y, tempo)

-16
